### Anthropic API

In [ ]:
# SetUp library
%pip install -U anthropic pydantic python-dotenv

import os, sys
import anthropic

print("Python:", sys.version)
print("anthropic SDK:", anthropic.__version__)
print("Kernel:", sys.executable)
print("Has ANTHROPIC_API_KEY?:", "ANTHROPIC_API_KEY" in os.environ)

In [ ]:
# Check API_KEY
import os

print("ANTHROPIC_API_KEY:", os.environ.get("ANTHROPIC_API_KEY", "Not set"))

### (1) messages.create：一般的な応答生成

In [ ]:
prompt = """以下の要件で簡潔に回答してください：
- 対象: Python学習者
- トピック: Anthropic APIのはじめ方
- 箇条書き3つ
"""
from anthropic import Anthropic

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

MODEL = "claude-3-5-haiku-20241022"

response = client.messages.create(
    model=MODEL,
    max_tokens=1000,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)  # テキスト抽出

#### (2) 構造化出力（Pydantic with JSON Schema）

In [ ]:
# 構造化出力（Pydantic）
from dotenv import load_dotenv
load_dotenv()

from anthropic import Anthropic
from pydantic import BaseModel, Field
from typing import List
import json, os

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
MODEL = os.getenv("ANTHROPIC_MODEL", "claude-3-5-haiku-20241022")

class TodoItem(BaseModel):
    title: str = Field(..., description="やることのタイトル")
    priority: int = Field(..., ge=1, le=5, description="1(低)〜5(高)")
    tags: List[str] = Field(default_factory=list)

class TodoPlan(BaseModel):
    owner: str
    items: List[TodoItem]

instruction = "あなたは秘書です。日本語で、学習計画のToDoを構造化して返してください。"
user_text = "明日はAnthropic APIを勉強、週末はQdrantを触る。優先度高いのは明日の学習。タグは['api','vector']"

# JSON Schemaを生成
def to_json_schema(model_cls):
    if hasattr(model_cls, "model_json_schema"):
        return model_cls.model_json_schema()
    return model_cls.schema()

schema = to_json_schema(TodoPlan)

# Anthropic APIで構造化出力を生成
response = client.messages.create(
    model=MODEL,
    max_tokens=1000,
    messages=[
        {
            "role": "user", 
            "content": f"""{instruction}
            
ユーザー入力: {user_text}

以下のJSONスキーマに厳密に従って、有効なJSONのみを出力してください：
{json.dumps(schema, ensure_ascii=False, indent=2)}

必ずJSONのみを返し、説明やマークダウンは一切含めないでください。"""
        }
    ],
    temperature=0
)

# JSONをパースしてPydanticモデルで検証
try:
    json_text = response.content[0].text.strip()
    # マークダウンのコードブロックがある場合は除去
    if json_text.startswith('```json'):
        json_text = json_text.split('```json')[1].split('```')[0].strip()
    elif json_text.startswith('```'):
        json_text = json_text.split('```')[1].split('```')[0].strip()
    
    data = json.loads(json_text)
    parsed_plan = TodoPlan.model_validate(data)
    print(parsed_plan)
except (json.JSONDecodeError, ValueError) as e:
    print(f"JSONパースエラー: {e}")
    print(f"生の応答: {response.content[0].text}")
except Exception as e:
    print(f"バリデーションエラー: {e}")
    print(f"パースされたデータ: {data}")

### (3) 注意：音声機能について

In [ ]:
# Anthropic APIには現在、音声生成機能（TTS）は含まれていません
# 音声機能が必要な場合は、以下のような代替手段を検討してください：
# 1. Google Cloud Text-to-Speech API
# 2. Azure Cognitive Services Speech
# 3. Amazon Polly
# 4. ElevenLabs API

print("注意: Anthropic APIには音声生成機能は含まれていません。")
print("音声機能が必要な場合は、専用の音声サービスを利用してください。")
print("")
print("例: Google Cloud Text-to-Speech APIの使用例")
print("pip install google-cloud-texttospeech")
print("")
print("from google.cloud import texttospeech")
print("client = texttospeech.TextToSpeechClient()")
print("# ... 音声生成処理")

### (4) 注意：音声認識機能について

In [ ]:
# Anthropic APIには現在、音声認識機能（STT）は含まれていません
# 音声認識が必要な場合は、以下のような代替手段を検討してください：
# 1. Google Cloud Speech-to-Text API
# 2. Azure Cognitive Services Speech
# 3. Amazon Transcribe
# 4. OpenAI Whisper API

print("注意: Anthropic APIには音声認識機能は含まれていません。")
print("音声認識が必要な場合は、専用の音声認識サービスを利用してください。")
print("")
print("例: Google Cloud Speech-to-Text APIの使用例")
print("pip install google-cloud-speech")
print("")
print("from google.cloud import speech")
print("client = speech.SpeechClient()")
print("# ... 音声認識処理")

### (5) 注意：音声翻訳機能について

In [ ]:
# Anthropic APIには現在、音声翻訳機能は含まれていません
# 音声翻訳が必要な場合は、以下のような代替手段を検討してください：
# 1. OpenAI Whisper API (音声→テキスト翻訳)
# 2. Google Translate API + 音声認識API
# 3. Azure Translator + 音声認識API

print("注意: Anthropic APIには音声翻訳機能は含まれていません。")
print("音声翻訳が必要な場合は、専用のサービスを利用してください。")
print("")
print("例: OpenAI Whisper APIで音声翻訳を行う場合")
print("pip install openai")
print("")
print("import openai")
print("client = openai.OpenAI()")
print("with open('audio.mp3', 'rb') as f:")
print("    translation = client.audio.translations.create(")
print("        model='whisper-1', file=f, response_format='text')")
print("    print(translation)")

### (6) 画像処理：Claude Vision

In [ ]:
# Claude Vision - 画像解析機能
# 注意: Anthropic APIには画像生成機能はありませんが、画像解析機能があります
import base64
from anthropic import Anthropic

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

# 画像生成は外部サービスを使用する必要があります
print("注意: Anthropic APIには画像生成機能は含まれていません。")
print("画像生成が必要な場合は、以下のようなサービスを利用してください：")
print("- DALL-E (OpenAI)")
print("- Midjourney")
print("- Stable Diffusion")
print("- Adobe Firefly")
print("")

# 代わりに、画像解析の例を示します
print("Claude Vision - 画像解析の例:")
print("")

# サンプル画像ファイル（5MB制限内のファイルを使用）
sample_image_path = "images/bmetal.png"  # 134KB - 5MB制限内

# 画像解析のサンプルコード:
try:
    with open(sample_image_path, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    
    response = client.messages.create(
        model="claude-3-5-sonnet-20241125",  # より新しいモデルを使用
        max_tokens=1000,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "この画像について詳しく説明してください。"
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_data
                        }
                    }
                ]
            }
        ]
    )
    print(response.content[0].text)
except FileNotFoundError:
    print("画像ファイルが見つかりません。適切な画像パスを指定してください。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

### (7) messages.create：詳細なメッセージ作成

In [ ]:
response = client.messages.create(
    model=MODEL,
    max_tokens=1000,
    system="あなたは有能なPythonメンターです。",
    messages=[
        {"role": "user", "content": "Anthropic APIでJSONの構造化出力を行うには？"},
    ],
    temperature=0.2
)

print(response.content[0].text)